In [1]:
import gc
import pandas as pd
import numpy as np
from sklearn import metrics, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor
import sys
import warnings
import itertools

if not sys.warnoptions:
    warnings.simplefilter("ignore")

path = "/home/darragh/avito/data/"
#path = '/Users/dhanley2/Documents/avito/data/'
#path = '/home/ubuntu/avito/data/'
#data_path = 'data/'

#lgb25 = pd.read_csv('../lgCV_2505.csv.gz', compression='gzip')
lgb02A = pd.read_csv(path+'../sub/lgCV_0206A.csv.gz', compression='gzip')
lgb27 = pd.read_csv(path+'../sub/lgCV_2705B.csv.gz', compression='gzip')
lgb31 = pd.read_csv(path+'../sub/lgCV_3105.csv.gz', compression='gzip')
lgb02 = pd.read_csv(path+'../sub/lgCV_0206.csv.gz', compression='gzip')
rnn =   pd.read_csv(path+'../sub/rnnCV_2805.csv.gz', compression='gzip')
rnn27 = pd.read_csv(path+'../sub/rnnCV_2705A.csv.gz', compression='gzip')
mlp =   pd.read_csv(path+'../sub/mlpCV_2505.csv.gz', compression='gzip')
truth = pd.read_csv(path+'train.csv.zip', compression='zip', parse_dates = ["activation_date"])
y =     truth['deal_probability'].values
truth.drop('deal_probability', 1)
test =  pd.read_csv(path+'test.csv.zip', compression='zip', parse_dates = ["activation_date"])
test['deal_probability']=float('NAN') 
truth = pd.concat([truth,test[truth.columns]],axis=0)

In [2]:
#rnn27trn = pd.read_pickle('../../features/y_pred_trn.pkl')
#rnn27tst = pd.read_csv('../rnndhsub_2705A.csv.gz', compression='gzip')
#rnn27 = pd.concat([rnn27tst, rnn27trn.reset_index().rename(columns = {0:'deal_probability'})], axis = 0)
#rnn27.to_csv('../rnnCV_2705A.csv.gz', compression='gzip')

In [3]:
#lgb25.rename(columns={'deal_probability': 'lgb25_preds' }, inplace=True)
lgb31.rename(columns={'deal_probability': 'lgb31_preds' }, inplace=True)
lgb27.rename(columns={'deal_probability': 'lgb27_preds' }, inplace=True)
lgb02.rename(columns={'deal_probability': 'lgb02_preds' }, inplace=True)
lgb02A.rename(columns={'deal_probability': 'lgb02A_preds' }, inplace=True)
rnn27.rename(columns={'deal_probability': 'rnn27_preds' }, inplace=True)
mlp.rename(columns={'deal_probability': 'mlp_preds' }, inplace=True)
preds_df = lgb27.merge(rnn, on='item_id')\
                .merge(mlp, on='item_id')\
                .merge(lgb31, on='item_id')\
                .merge(lgb02, on='item_id')\
                .merge(lgb02A, on='item_id')\
                .merge(rnn27, on='item_id')\
                .merge(truth, on='item_id',how='left')

In [4]:
#preds_df['difference_l02_r']=preds_df['lgb02A_preds'] - preds_df['rnn_preds']
#preds_df['difference_l02_r1']=preds_df['lgb02_preds'] - preds_df['rnn27_preds']
#preds_df['difference_l02_m']=preds_df['lgb02_preds'] - preds_df['mlp_preds']
#preds_df['difference_r_m']=preds_df['rnn27_preds'] - preds_df['mlp_preds']
#preds_df['difference_l02_l02A']=preds_df['lgb02_preds'] - preds_df['lgb02A_preds']
#preds_df['difference_l27_l25']=preds_df['lgb27_preds'] - preds_df['lgb25_preds']
pred_cols = [col for col in preds_df.columns if ('_preds' in col) \
             and ('lgb27' not in col) and ('lgb02_' not in col)]

preds_df['preds_sum'] = preds_df[pred_cols].sum(axis=1)
pred_cols

['rnn_preds', 'mlp_preds', 'lgb31_preds', 'lgb02A_preds', 'rnn27_preds']

In [5]:
preds_df['price'].fillna(-1,inplace=True)
preds_df['max'] = np.max(np.array([preds_df[col] for col in pred_cols]),axis=0)
preds_df['min'] = np.min(np.array([preds_df[col] for col in pred_cols]),axis=0)
preds_df['avg'] = np.mean(np.array([preds_df[col] for col in pred_cols]),axis=0)
preds_df['std'] = np.std(np.array([preds_df[col] for col in pred_cols]),axis=0)

In [6]:
import itertools
for p1, p2 in itertools.combinations(pred_cols, 2):
    print('difference_%s__%s'%(p1,p2))
    preds_df['difference_%s__%s'%(p1,p2)] = preds_df[p2] - preds_df[p1]

difference_rnn_preds__mlp_preds
difference_rnn_preds__lgb31_preds
difference_rnn_preds__lgb02A_preds
difference_rnn_preds__rnn27_preds
difference_mlp_preds__lgb31_preds
difference_mlp_preds__lgb02A_preds
difference_mlp_preds__rnn27_preds
difference_lgb31_preds__lgb02A_preds
difference_lgb31_preds__rnn27_preds
difference_lgb02A_preds__rnn27_preds


In [7]:

gc.collect()

132

In [8]:
preds_df.head()

,item_id,lgb27_preds,rnn_preds,mlp_preds,lgb31_preds,lgb02_preds,lgb02A_preds,Unnamed: 0,rnn27_preds,user_id,...,difference_rnn_preds__mlp_preds,difference_rnn_preds__lgb31_preds,difference_rnn_preds__lgb02A_preds,difference_rnn_preds__rnn27_preds,difference_mlp_preds__lgb31_preds,difference_mlp_preds__lgb02A_preds,difference_mlp_preds__rnn27_preds,difference_lgb31_preds__lgb02A_preds,difference_lgb31_preds__rnn27_preds,difference_lgb02A_preds__rnn27_preds
0,b912c3c6a6ad,0.091656,0.069891,0.091223,0.083219,0.090855,0.109617,0,0.068331,e00f8ff2eaf9,...,0.021332,0.013328,0.039726,-0.001560,-0.008004,0.018394,-0.022892,0.026398,-0.014888,-0.041287
1,2dac0150717d,0.147801,0.075968,0.211844,0.157973,0.119609,0.160045,1,0.070638,39aeb48f0017,...,0.135876,0.082005,0.084077,-0.005330,-0.053871,-0.051800,-0.141206,0.002071,-0.087335,-0.089406
2,ba83aefab5dc,0.186261,0.167167,0.236579,0.227048,0.258980,0.241336,2,0.161924,91e2f88dd6e3,...,0.069412,0.059881,0.074169,-0.005243,-0.009531,0.004757,-0.074655,0.014288,-0.065124,-0.079412
3,02996f1dd2ea,0.241699,0.260876,0.383699,0.231447,0.284130,0.336456,3,0.278804,bf5cccea572d,...,0.122823,-0.029428,0.075580,0.017928,-0.152251,-0.047243,-0.104895,0.105008,0.047356,-0.057652
4,7c90be56d2ab,0.407423,0.431520,0.420657,0.412563,0.430581,0.421989,4,0.350284,ef50846afc0b,...,-0.010863,-0.018957,-0.009531,-0.081235,-0.008094,0.001332,-0.070373,0.009426,-0.062278,-0.071704


### Train scores

In [9]:
pred_cols = [col for col in preds_df.columns if ('preds' in col) and ('difference' not in col)]
pred_cols

['lgb27_preds',
 'rnn_preds',
 'mlp_preds',
 'lgb31_preds',
 'lgb02_preds',
 'lgb02A_preds',
 'rnn27_preds',
 'preds_sum']

In [10]:
idx = preds_df['deal_probability']==preds_df['deal_probability']
print(idx.value_counts())
for col in [c for c in preds_df.columns if ('_preds' in c)  and ('difference' not in c) ]:
    print('RMSE %s: '%(col), np.sqrt(metrics.mean_squared_error(preds_df['deal_probability'][idx].values, preds_df[col][idx].values)))

True     1503424
False     508438
Name: deal_probability, dtype: int64
RMSE lgb27_preds:  0.21681527582821059
RMSE rnn_preds:  0.21771476573780568
RMSE mlp_preds:  0.2187544351068867
RMSE lgb31_preds:  0.21681135578306754
RMSE lgb02_preds:  0.21625386267187377
RMSE lgb02A_preds:  0.21627136604335995
RMSE rnn27_preds:  0.21698040871120608


In [11]:
foldls = [["2017-03-15", "2017-03-16", "2017-03-17"], \
       ["2017-03-18", "2017-03-19", "2017-03-20"], \
       ["2017-03-21", "2017-03-22", "2017-03-23"], \
       ["2017-03-24", "2017-03-25", "2017-03-26"], \
        ["2017-03-27", "2017-03-28", "2017-03-29", \
            "2017-03-30", "2017-03-31", "2017-04-01", \
            "2017-04-02", "2017-04-03","2017-04-07"]]
foldls = [[pd.to_datetime(d) for d in f] for f in foldls]
preds_df['fold'] = -1
for t, fold in enumerate(foldls):
    preds_df['fold'][preds_df.activation_date.isin(fold)] = t

In [12]:
scores = []
for col in [c for c in preds_df.columns if ('_preds' in c)  and ('difference' not in c) ]:
    lstmp = [col]
    for i in range(5):
        idx = preds_df['fold']==i
        lstmp.append(np.sqrt(metrics.mean_squared_error(preds_df['deal_probability'][idx].values, \
                                                        preds_df[col][idx].values)))
    scores.append(lstmp)
pd.DataFrame(scores, columns = ['Model']+['Fold%s'%(i) for i in range(5)])

,Model,Fold0,Fold1,Fold2,Fold3,Fold4
0,lgb27_preds,0.216848,0.217096,0.217923,0.217597,0.213692
1,rnn_preds,0.218756,0.218966,0.215764,0.219554,0.214747
2,mlp_preds,0.219660,0.220088,0.216672,0.220347,0.216385
3,lgb31_preds,0.216537,0.216643,0.219230,0.217169,0.213430
4,lgb02_preds,0.216371,0.216447,0.217354,0.217032,0.213158
5,lgb02A_preds,0.216081,0.216051,0.218825,0.216562,0.212741
6,rnn27_preds,0.218097,0.218115,0.214982,0.218651,0.214379


### Correlations in test and train

In [13]:
# Test correlation
preds_df[~preds_df['deal_probability'].isnull()][[c for c in preds_df.columns if ('_preds' in c)  and ('difference' not in c) ]].corr()

,lgb27_preds,rnn_preds,mlp_preds,lgb31_preds,lgb02_preds,lgb02A_preds,rnn27_preds
lgb27_preds,1.000000,0.915952,0.919171,0.990750,0.988682,0.983143,0.916804
rnn_preds,0.915952,1.000000,0.929710,0.910602,0.913307,0.911443,0.985360
mlp_preds,0.919171,0.929710,1.000000,0.914307,0.916343,0.913600,0.925010
lgb31_preds,0.990750,0.910602,0.914307,1.000000,0.991984,0.986851,0.914204
lgb02_preds,0.988682,0.913307,0.916343,0.991984,1.000000,0.987434,0.916726
lgb02A_preds,0.983143,0.911443,0.913600,0.986851,0.987434,1.000000,0.914911
rnn27_preds,0.916804,0.985360,0.925010,0.914204,0.916726,0.914911,1.000000


In [14]:
# Train correlation
preds_df[preds_df['deal_probability'].isnull()][[c for c in preds_df.columns if ('_preds' in c)  and ('difference' not in c) ]].corr()

,lgb27_preds,rnn_preds,mlp_preds,lgb31_preds,lgb02_preds,lgb02A_preds,rnn27_preds
lgb27_preds,1.000000,0.930288,0.924283,0.991439,0.989502,0.984742,0.930692
rnn_preds,0.930288,1.000000,0.939951,0.927835,0.927867,0.928496,0.988198
mlp_preds,0.924283,0.939951,1.000000,0.922413,0.921927,0.922121,0.935069
lgb31_preds,0.991439,0.927835,0.922413,1.000000,0.993168,0.988043,0.931468
lgb02_preds,0.989502,0.927867,0.921927,0.993168,1.000000,0.989577,0.931447
lgb02A_preds,0.984742,0.928496,0.922121,0.988043,0.989577,1.000000,0.932071
rnn27_preds,0.930692,0.988198,0.935069,0.931468,0.931447,0.932071,1.000000


In [15]:
cols = [c for c in preds_df.columns if '_preds' in c]
cols += [c for c in preds_df.columns if 'difference' in c]
cols += ['price', 'max', 'min', 'avg', 'std']
categories = ['region', 'param_1', 'parent_category_name', 'category_name', 'param_2' ]
cols += categories
cols = list(set(cols))

In [16]:
for col in categories:
    preds_df[col] = LabelEncoder().fit_transform(preds_df[col].fillna("0"))
train_df = preds_df[~preds_df['deal_probability'].isnull()]
test_df = preds_df[preds_df['deal_probability'].isnull()]

In [17]:
n_estimators = 4000
train_X, valid_X, train_y, valid_y = train_test_split(train_df[cols], y, train_size=.8, random_state=12345)
eval_set = [(train_X,train_y),(valid_X,valid_y)]

In [18]:
len(train_X)

1202739

In [19]:
%%time
clf = LGBMRegressor(n_estimators=n_estimators, 
                    max_depth=-1, 
                    feature_fraction= 0.5,
                    num_leaves=32, 
                    learning_rate=.005)#, device='gpu')
clf.fit(train_X, train_y, early_stopping_rounds=80, 
        eval_set=eval_set, eval_metric='rmse', verbose=100, 
        categorical_feature=categories)
# valid_1's rmse: 0.213702

Training until validation scores don't improve for 80 rounds.
[100]	training's rmse: 0.232076	valid_1's rmse: 0.23232
[200]	training's rmse: 0.220682	valid_1's rmse: 0.221054
[300]	training's rmse: 0.216174	valid_1's rmse: 0.216674
[400]	training's rmse: 0.21436	valid_1's rmse: 0.214981
[500]	training's rmse: 0.213582	valid_1's rmse: 0.214318
[600]	training's rmse: 0.213207	valid_1's rmse: 0.214049
[700]	training's rmse: 0.212989	valid_1's rmse: 0.21393
[800]	training's rmse: 0.212838	valid_1's rmse: 0.21387
[900]	training's rmse: 0.212719	valid_1's rmse: 0.213837
[1000]	training's rmse: 0.212618	valid_1's rmse: 0.213817
[1100]	training's rmse: 0.212524	valid_1's rmse: 0.2138
[1200]	training's rmse: 0.212441	valid_1's rmse: 0.21379
[1300]	training's rmse: 0.212363	valid_1's rmse: 0.213781
[1400]	training's rmse: 0.212288	valid_1's rmse: 0.213773
[1500]	training's rmse: 0.212219	valid_1's rmse: 0.213766
[1600]	training's rmse: 0.212156	valid_1's rmse: 0.213759
[1700]	training's rmse: 0.

In [20]:
sorted(zip(clf.feature_importances_, train_X.columns ),key=lambda x: -x[0])

[(19488, 'param_1'),
 (11679, 'param_2'),
 (11065, 'region'),
 (5817, 'category_name'),
 (5133, 'difference_rnn_preds__rnn27_preds'),
 (4332, 'avg'),
 (4255, 'rnn27_preds'),
 (4222, 'lgb02_preds'),
 (4014, 'difference_lgb31_preds__lgb02A_preds'),
 (3940, 'mlp_preds'),
 (3891, 'lgb02A_preds'),
 (3700, 'difference_mlp_preds__rnn27_preds'),
 (3562, 'min'),
 (3550, 'difference_rnn_preds__mlp_preds'),
 (3410, 'price'),
 (3338, 'difference_mlp_preds__lgb31_preds'),
 (3190, 'difference_lgb31_preds__rnn27_preds'),
 (3076, 'difference_mlp_preds__lgb02A_preds'),
 (3045, 'difference_rnn_preds__lgb02A_preds'),
 (2959, 'difference_rnn_preds__lgb31_preds'),
 (2848, 'lgb27_preds'),
 (2792, 'difference_lgb02A_preds__rnn27_preds'),
 (2533, 'rnn_preds'),
 (2510, 'std'),
 (2425, 'max'),
 (2409, 'lgb31_preds'),
 (817, 'parent_category_name')]

In [21]:
n_estimators = 4000
train_X = train_df[cols]
train_y = y
eval_set = [(train_X,train_y)]
len(train_X)

1503424

In [22]:
%%time
clf = LGBMRegressor(n_estimators=n_estimators, 
                    max_depth=-1, 
                    feature_fraction= 0.5,
                    num_leaves=32, 
                    learning_rate=.005)#, device='gpu')

clf.fit(train_X, train_y, early_stopping_rounds=80, 
        eval_set=eval_set, eval_metric='rmse', verbose=100, 
        categorical_feature=categories)

Training until validation scores don't improve for 80 rounds.
[100]	training's rmse: 0.232129
[200]	training's rmse: 0.22076
[300]	training's rmse: 0.216276
[400]	training's rmse: 0.214483
[500]	training's rmse: 0.213723
[600]	training's rmse: 0.213364
[700]	training's rmse: 0.213161
[800]	training's rmse: 0.213023
[900]	training's rmse: 0.212916
[1000]	training's rmse: 0.212826
[1100]	training's rmse: 0.212743
[1200]	training's rmse: 0.212669
[1300]	training's rmse: 0.212599
[1400]	training's rmse: 0.212534
[1500]	training's rmse: 0.212472
[1600]	training's rmse: 0.212417
[1700]	training's rmse: 0.212362
[1800]	training's rmse: 0.212311
[1900]	training's rmse: 0.212262
[2000]	training's rmse: 0.212212
[2100]	training's rmse: 0.212163
[2200]	training's rmse: 0.212114
[2300]	training's rmse: 0.212068
[2400]	training's rmse: 0.212021
[2500]	training's rmse: 0.211973
[2600]	training's rmse: 0.211928
[2700]	training's rmse: 0.211883
[2800]	training's rmse: 0.211839
[2900]	training's rmse: 

In [23]:
test_df['deal_probability'] = clf.predict(test_df[cols])
test_df['deal_probability'] = np.clip(test_df['deal_probability'], .0001, .9999)
test_df[['item_id', 'deal_probability']].to_csv('../lgbbsub_0406BL2.csv.gz', compression='gzip', index=False, header=True)

In [24]:
test_df[['item_id', 'deal_probability']].head()

,item_id,deal_probability
1503424,6544e41a8817,0.447456
1503425,65b9484d670f,0.143087
1503426,8bab230b2ecd,0.130201
1503427,8e348601fefc,0.059738
1503428,8bd2fe400b89,0.206799
